# Scalbale Recommendation Retrieval with Locality Sensitive Hashing

From the previous example, we can see that the cost of exhaustive search is linear to the number of items, i.e., $n$ and number of features, i.e., $d$. 

In this part, we will practice to use Locality Sensitive Hashing to speed up the recommendation retrieval task. 

In [1]:
%matplotlib inline
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from utils.lsh import *
from utils.load_data import *
from utils.pmf import *
from utils.evaluation import *

In [2]:
#load train/test data and trained pmf model
pmf_mvl = pickle.load(open('pmf_mvl1m.model', 'rb'))
train   = pickle.load(open('train_data', 'rb'))
test    = pickle.load(open('test_data', 'rb'))

The function defined below measure precision and recall value of the top-k recommendation list returned to each user by the model. The result is average over number of users in the testing set.

## Performance of Linear Scanning Solution

We first measure the precision@10 and recall@10

In [5]:
#number of recommendations
topK = 10

#construct the recommendation list
inv_lst = np.unique(test[:, 0]).astype(int)
rec_list = {}
for inv in inv_lst:
    if rec_list.get(inv, None) is None:
        rec_list[inv] = np.argsort(pmf_mvl.predict(inv))[-topK:]

In [6]:
linear_prec, linear_recall = evaluate_topK(test, rec_list, topK)
print('linear_prec@{0} \t linear_recall@{0}'.format(topK))
print('{0}\t{1}'.format(linear_prec, linear_recall))

linear_prec@10 	 linear_recall@10
0.07596026490066361	0.02246526738773978


## Locality Sensitive Hashing

One of the most popular search protocal using Locality Sensitive Hashing structure is Hashtable look-up (illustrated below).  

<img src="resources/images/lsh_retrieval.png" width="600">

### Performances without post-processing

In this experiment, we immediately build LSH index on the output of PMF algorithm. You should expect to see the precision and recall degeneration as compared to those of linear scanning solution. Here, we report three values here:

1. relative_prec@10 = $\frac{\text{precision@10 of LSH Indexing}}{\text{precision@10 of linearning scanning}}$ 
&nbsp;
&nbsp;
&nbsp;  
&nbsp;
2. relative_rec@10    = $\frac{\text{recall@10 of LSH Indexing}}{\text{recall@10 of linearning scanning}}$
&nbsp;
&nbsp;
&nbsp;  
&nbsp;
3. touched = $\frac{\text{Average number of investigated items by LSH}}{\text{Total number of items}}$

In [ ]:
topK = 10
b_vals = [4, 6, 8, 10]
L_vals = [10, 20, 30]

queries = pmf_mvl.w_User
data    = pmf_mvl.w_Item

print('#table\t #bit \t relative_prec@{0} \t relative_recall@{0} \t touched'.format(topK))
for nt in L_vals:
    print('-----------------------------------------------------------------------')
    for b in b_vals: 
        prec, recall, touched = evalate_LSHTopK(test, data, -queries, CosineHashFamily, nt, b, dot, topK)
        print("{0}\t{1}\t{2}\t{3}\t{4}".format(nt, b, prec/linear_prec, recall/linear_recall, touched)) 


#table	 #bit 	 relative_prec@10 	 relative_recall@10 	 touched
-----------------------------------------------------------------------
10	4	0.5921970357454026	0.5254602505803535	0.5487593042755691
10	6	0.4457279860505477	0.3726053930829359	0.1794293628948087
10	8	0.2883609415867385	0.2011188303742779	0.06183592644030939
10	10	0.13927637314733962	0.08850741285891302	0.022793988302957098
-----------------------------------------------------------------------


### Performances with post-processing Xbox transformation

Now, before building LSH index, we first apply the Xbox transformation for both user and item vectors. This original maximum inner product search on the original representation becomes the maximum cosine similarity search on the new representation.

\begin{equation}
P(y_i) = [y_i, \sqrt{M^2 - ||y_i||^2}];  Q(x_u) = [x_u, 0]
\end{equation}


In [ ]:
#apply Xbox transformation
topK = 10
b_vals = [4, 6, 8, 10]
L_vals = [10, 20, 30]

queries = pmf_mvl.w_User
data    = pmf_mvl.w_Item

M = np.linalg.norm(data, axis=1)
max_norm = max(M)
q_norm = np.sqrt((queries * queries).sum(axis=1))
n_queries = queries/q_norm.reshape(queries.shape[0], 1)

n_data = np.concatenate((data, np.sqrt(max_norm**2 - pow(M, 2)).reshape(data.shape[0], -1)), axis = 1)
n_data = n_data/max_norm # normalized data vectors
n_queries = np.concatenate((n_queries, np.zeros((n_queries.shape[0], 1))), axis = 1)

print('Done X-box transformation!')
print('#table\t #bit \t relative_prec@{0} \t relative_recall@{0} \t touched'.format(topK))
for nt in L_vals:
    print('-----------------------------------------------------------------------')
    for b in b_vals: 
        prec, recall, touched = evalate_LSHTopK(test, n_data, -n_queries, CosineHashFamily, nt, b, dot, topK)
        print("{0}\t{1}\t{2}\t{3}\t{4}".format(nt, b, prec/linear_prec, recall/linear_recall, touched)) 
